In [1]:
!pip install transformers datasets chess optuna fsspec==2024.10.0 --upgrade

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached datasets-3.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached datasets-3.0.1-py3-none-any.whl.metadata (20 kB)
  Using cached datasets-3.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached datasets-2.20.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl.metadata (3.6 kB)
  Using cached datasets-2.19.2-py3-none-any.whl.metadata (19 kB)
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-2.19.1-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-2.19.0-py3-none-any.whl.metadat

In [2]:
# Import libraries
import pandas as pd
import chess
import chess.engine
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os
from google.colab import drive
import optuna
import torch
import time  # Added import for time tracking

##############################################
# Google Drive Setup
##############################################

# Mount Google Drive
drive.mount('/content/drive')

# Define persistent output directory
output_dir = "/content/drive/MyDrive/chess_model_results"

##############################################
# Step 1: Data Preparation
##############################################

def load_and_prepare_data(file_path):
    """Load and preprocess the chess dataset."""
    chess_data = pd.read_csv(file_path)
    grouped_games = chess_data.groupby('game_id')
    text_data = []

    for _, group in grouped_games:
        game_moves = group['notation'].tolist()
        text_data.append(' '.join(game_moves))

    dataset = Dataset.from_dict({'text': text_data})
    return dataset.train_test_split(test_size=0.1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
##############################################
# Step 2: Tokenization
##############################################

def tokenize_dataset(dataset, tokenizer):
    """Tokenize the dataset and prepare labels."""
    def tokenize_function(examples):
        tokenized = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
        tokenized['labels'] = tokenized['input_ids'].copy()
        return tokenized

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return tokenized_dataset

In [4]:
##############################################
# Step 3: Hyperparameter Optimization
##############################################

def objective(trial, dataset, tokenizer):
    """Define the objective for hyperparameter tuning."""
    model = T5ForConditionalGeneration.from_pretrained("t5-small").to("cuda" if torch.cuda.is_available() else "cpu")

    # Fixed hyperparameters
    batch_size = 16
    learning_rate = 5e-4
    num_train_epochs = 20

    # Training arguments
    training_args = TrainingArguments(
        output_dir="/tmp/trial_results",
        evaluation_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=batch_size,
        learning_rate=learning_rate,
        num_train_epochs=num_train_epochs,
        logging_dir="/tmp/logs",
        logging_steps=10,
        disable_tqdm=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
    )

    # Train model and return evaluation loss
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]

In [5]:
##############################################
# Step 4: Model Training with Time Estimation per Epoch
##############################################

def train_model_with_best_params(dataset, tokenizer, output_dir, best_params, run_path=None):
    """Train the model using the best hyperparameters and optionally resume from a checkpoint."""
    last_checkpoint = None

    if run_path:
        last_checkpoint = run_path
        print(f"Resuming from specified checkpoint: {last_checkpoint}")
    else:
        if os.path.exists(output_dir):
            checkpoints = [
                os.path.join(output_dir, d)
                for d in os.listdir(output_dir)
                if os.path.isdir(os.path.join(output_dir, d)) and d.startswith("checkpoint-")
            ]
            if checkpoints:
                last_checkpoint = sorted(checkpoints, key=os.path.getmtime)[-1]
                print(f"Found checkpoint: {last_checkpoint}")

    model = T5ForConditionalGeneration.from_pretrained(last_checkpoint if last_checkpoint else "t5-small").to("cuda" if torch.cuda.is_available() else "cpu")

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,
        per_device_train_batch_size=best_params["batch_size"],
        learning_rate=best_params["learning_rate"],
        num_train_epochs=best_params["num_train_epochs"],
        logging_dir="./logs",
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
    )

    print("Starting training...")

    total_time = 0  # Initialize total time variable
    for epoch in range(best_params["num_train_epochs"]):
        start_time = time.time()  # Record start time of the epoch
        print(f"Epoch {epoch + 1}/{best_params['num_train_epochs']}...")

        trainer.train(resume_from_checkpoint=last_checkpoint)

        epoch_time = time.time() - start_time  # Calculate time for this epoch
        total_time += epoch_time  # Add to total time

        print(f"Epoch {epoch + 1} completed in {epoch_time:.2f} seconds")

    print(f"Training complete. Total time: {total_time:.2f} seconds.")

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Model and tokenizer saved to {output_dir}")

    return model

In [6]:
##############################################
# Step 5: Chatbot Functionality
##############################################

def chess_advisor(input_moves, model, tokenizer):
    """Generate a response suggesting the next move."""
    model.eval()
    input_text = ' '.join(input_moves) + ' next move:'
    inputs = tokenizer.encode_plus(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    input_ids = inputs["input_ids"].to("cuda" if torch.cuda.is_available() else "cpu")
    attention_mask = inputs["attention_mask"].to("cuda" if torch.cuda.is_available() else "cpu")

    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=512,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the next move from the generated text
    next_move = "[No Move Suggested]"
    if 'next move:' in generated_text:
        try:
            next_move = generated_text.split('next move:')[1].strip().split()[0]
        except (IndexError, ValueError):
            pass
    return next_move

In [7]:
##############################################
# Step 6: Accuracy Calculation
##############################################

def calculate_accuracy(test_cases, model, tokenizer):
    """Calculate accuracy of the model based on test cases."""
    correct_predictions = 0
    for moves, expected_move in test_cases:
        predicted_move = chess_advisor(moves, model, tokenizer)
        print(f"Input moves: {moves} - Expected move: {expected_move}")
        print(f"Predicted move: {predicted_move}")

        if predicted_move == expected_move:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_cases) * 100
    print(f"Model Accuracy: {accuracy:.2f}%")
    return accuracy

In [8]:
##############################################
# Step 7: Inference
##############################################

def run_pipeline(file_path, output_dir):
    """Complete pipeline for loading data, tuning, training, and suggesting moves."""
    # Load and prepare data
    dataset = load_and_prepare_data(file_path)

    # Load tokenizer
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    tokenizer.pad_token = tokenizer.eos_token

    # Tokenize dataset
    tokenized_dataset = tokenize_dataset(dataset, tokenizer)

    # Hyperparameter tuning
    best_params = {
        "batch_size": 16,
        "learning_rate": 5e-4,
        "num_train_epochs": 20,
    }

   # Train model with best hyperparameters
    model = train_model_with_best_params(tokenized_dataset, tokenizer, output_dir, best_params)

    # Test chatbot with various scenarios
    test_cases = [
        (["e4", "e5", "Nf3", "Nc6"], "e4"),  # Open game
        (["d4", "d5", "c4", "e6"], "c4"),  # Queen's Gambit
        (["Nf3", "d5", "g3"], "g3"),  # King's Indian Attack
        (["e4", "c5", "Nf3", "d6"], "d6"),  # Sicilian Defense
        (["c4", "e5", "g3"], "g3"),  # English Opening
        (["e4", "e6", "d4", "d5"], "d4"),  # French Defense
        (["d4", "Nf6", "c4", "g6"], "c4"),  # King's Indian Defense
        (["e4", "c6", "d4", "d5"], "d4"),  # Caro-Kann Defense
        (["g3", "d5", "Bg2", "Nf6"], "Bg2"),  # Fianchetto Opening
        (["e4", "c5", "c3"], "c3")  # Alapin Variation
    ]

    for moves, expected_move in test_cases:
        print(f"Input moves: {moves}")
        next_move = chess_advisor(moves, model, tokenizer)
        print(f"Suggested next move: {next_move}")

    # Calculate and display accuracy
    calculate_accuracy(test_cases, model, tokenizer)

In [9]:

##############################################
# Step 8: Run the Pipeline
##############################################

# Update file_path with your dataset location
file_path = "carlsen_games_moves.csv"  # Replace with your dataset file path

run_pipeline(file_path, output_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/11158 [00:00<?, ? examples/s]

Map:   0%|          | 0/1240 [00:00<?, ? examples/s]

Found checkpoint: /content/drive/MyDrive/chess_model_results/checkpoint-111580


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss


Epoch 19 completed in 1.74 seconds
Epoch 20/20...


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Epoch,Training Loss,Validation Loss


Epoch 20 completed in 1.69 seconds
Training complete. Total time: 44.32 seconds.
Model and tokenizer saved to /content/drive/MyDrive/chess_model_results
Input moves: ['e4', 'e5', 'Nf3', 'Nc6']
Suggested next move: [No Move Suggested]
Input moves: ['d4', 'd5', 'c4', 'e6']
Suggested next move: [No Move Suggested]
Input moves: ['Nf3', 'd5', 'g3']
Suggested next move: [No Move Suggested]
Input moves: ['e4', 'c5', 'Nf3', 'd6']
Suggested next move: [No Move Suggested]
Input moves: ['c4', 'e5', 'g3']
Suggested next move: [No Move Suggested]
Input moves: ['e4', 'e6', 'd4', 'd5']
Suggested next move: [No Move Suggested]
Input moves: ['d4', 'Nf6', 'c4', 'g6']
Suggested next move: [No Move Suggested]
Input moves: ['e4', 'c6', 'd4', 'd5']
Suggested next move: [No Move Suggested]
Input moves: ['g3', 'd5', 'Bg2', 'Nf6']
Suggested next move: [No Move Suggested]
Input moves: ['e4', 'c5', 'c3']
Suggested next move: [No Move Suggested]
Input moves: ['e4', 'e5', 'Nf3', 'Nc6'] - Expected move: e4
Predicte

In [10]:
import re

##############################################
# Step 9: Interactive Chatbot with Dynamic Input Parsing
##############################################

def extract_moves(user_input):
    """Extract chess moves dynamically from user input."""
    # Updated regex to handle moves with punctuation and spaces
    moves_pattern = r'\b(?:[KQBNR]?[a-h]?[1-8]?[x-]?[a-h][1-8]|O-O(?:-O)?)\b'
    # Find all matches, strip spaces, and return as a list
    moves = re.findall(moves_pattern, user_input)
    return [move.strip() for move in moves if move.strip()]

def interactive_chatbot(model, tokenizer):
    """Interactive chatbot for chess move prediction with dynamic input parsing."""
    print("Chatbot: Good day! I will try my best to predict chess moves based on your setting!")
    print("Type 'exit' to end the chat.")

    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Chatbot: Thank you for chatting! Goodbye!")
            break

        # Extract moves from the user's input dynamically
        moves = extract_moves(user_input)
        if moves:
            print(f"Chatbot: Analyzing the moves: {moves}")
            # Predict the next move
            next_move = chess_advisor(moves, model, tokenizer)
            print(f"Chatbot: I can predict that the best next move is {next_move}.")
        else:
            print("Chatbot: Sorry, I couldn't extract valid moves. Please provide a valid sequence of moves.")

##############################################
# Step 10: Run Interactive Chatbot
##############################################

# Ensure the pipeline is executed and the model/tokenizer are ready before running the chatbot.
file_path = "carlsen_games_moves.csv"  # Replace with your dataset file path

# Load and prepare data
dataset = load_and_prepare_data(file_path)

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
tokenized_dataset = tokenize_dataset(dataset, tokenizer)

# Hyperparameter tuning and training
best_params = {
    "batch_size": 16,
    "learning_rate": 5e-4,
    "num_train_epochs": 20,
}
model = train_model_with_best_params(tokenized_dataset, tokenizer, output_dir, best_params)

Map:   0%|          | 0/11158 [00:00<?, ? examples/s]

Map:   0%|          | 0/1240 [00:00<?, ? examples/s]

Found checkpoint: /content/drive/MyDrive/chess_model_results/checkpoint-111580


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training...
Epoch 1/20...


...

Epoch,Training Loss,Validation Loss


Epoch 19 completed in 2.46 seconds
Epoch 20/20...


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Epoch,Training Loss,Validation Loss


Epoch 20 completed in 1.70 seconds
Training complete. Total time: 40.42 seconds.
Model and tokenizer saved to /content/drive/MyDrive/chess_model_results


In [11]:
# Run the chatbot
interactive_chatbot(model, tokenizer)

Chatbot: Good day! I will try my best to predict chess moves based on your setting!
Type 'exit' to end the chat.
User: Predict the next move if the moves are e7, Nf6, c4, Nc6, g3, e6, and Nc3
Chatbot: Analyzing the moves: ['e7', 'Nf6', 'c4', 'Nc6', 'g3', 'e6', 'Nc3']
Chatbot: I can predict that the best next move is [No Move Suggested].
User: exit
Chatbot: Thank you for chatting! Goodbye!
